## 1- Data Ingestion
Source : https://covidtracking.com/data/api

Data of covid 19 cases in USA

In [0]:
import requests
import csv
from pyspark.sql import SparkSession
from pyspark.sql.types import *

url = 'https://api.covidtracking.com/v1/states/daily.csv'
response = requests.get(url)

csv_data = csv.reader(response.text.strip().split('\n'))

header = next(csv_data)

fields = [StructField(field_name, StringType(), True) for field_name in header]
schema = StructType(fields)

spark = SparkSession.builder.appName("COVID Tracking Data - Data Ingestion").getOrCreate()

df = spark.createDataFrame(csv_data, schema=schema)
destination_path = '/FileStore/tables/Covid-USA/raw/daily_cases_raw'
df.write.mode('overwrite').format("parquet").save(destination_path)

spark.stop()